In [ ]:
# Install necessary libraries
!pip install yfinance pandas_ta beautifulsoup4 lxml

In [26]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import datetime
import warnings
import concurrent.futures
import requests
import time

warnings.filterwarnings('ignore')

# Define the time period for historical data (2 years for sufficient MA200 data)
end_date = datetime.datetime.today()
start_date = end_date - datetime.timedelta(days=730)  # Last 2 years

# Predefined and Cleaned list of NSE and BSE tickers across large-cap, mid-cap, and small-cap
# Removed duplicates and corrected suffixes
stock_tickers = [
    # Large-cap NSE
    'RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', 'INFY.NS',
    'ICICIBANK.NS', 'HINDUNILVR.NS', 'KOTAKBANK.NS', 'LT.NS',
    'SBILIFE.NS', 'BAJFINANCE.NS', 'ITC.NS', 'HDFC.NS',
    'AXISBANK.NS', 'ASIANPAINT.NS', 'BAJAJFINSV.NS',

    # Mid-cap NSE
    'ADANIPORTS.NS', 'BAJAJ-AUTO.NS', 'CIPLA.NS', 'DRREDDY.NS',
    'DIVISLAB.NS', 'JSWSTEEL.NS', 'HCLTECH.NS', 'ULTRACEMCO.NS',
    'HDFCLIFE.NS', 'TECHM.NS', 'TITAN.NS', 'M&M.NS',
    'MARUTI.NS', 'ZEEL.NS', 'HEROMOTOCO.NS',

    # Small-cap NSE
    'MANAPPURAM.NS', 'SBIN.NS', 'POWERGRID.NS', 'NTPC.NS',
    'VEDL.NS', 'POLYCAB.NS', 'GLENMARK.NS', 'SPCL.NS',
    'TORNTPHARM.NS', 'TATAPOWER.NS', 'BALRAMCHIN.NS',
    'BSOFT.NS', 'KPRMILL.NS', 'JKTYRE.NS',

    # Large-cap BSE
    'RELIANCE.BO', 'TCS.BO', 'HDFCBANK.BO', 'INFY.BO',
    'ICICIBANK.BO', 'HINDUNILVR.BO', 'KOTAKBANK.BO', 'LT.BO',
    'SBILIFE.BO', 'BAJFINANCE.BO', 'ITC.BO', 'HDFC.BO',
    'AXISBANK.BO', 'ASIANPAINT.BO', 'BAJAJFINSV.BO',

    # Mid-cap BSE
    'ADANIPORTS.BO', 'BAJAJ-AUTO.BO', 'CIPLA.BO', 'DRREDDY.BO',
    'DIVISLAB.BO', 'JSWSTEEL.BO', 'HCLTECH.BO', 'ULTRACEMCO.BO',
    'HDFCLIFE.BO', 'TECHM.BO', 'TITAN.BO', 'M&M.BO',
    'MARUTI.BO', 'ZEEL.BO', 'HEROMOTOCO.BO',

    # Small-cap BSE
    'MANAPPURAM.BO', 'SBIN.BO', 'POWERGRID.BO', 'NTPC.BO',
    'VEDL.BO', 'POLYCAB.BO', 'GLENMARK.BO', 'SPCL.BO',
    'TORNTPHARM.BO', 'TATAPOWER.BO', 'BALRAMCHIN.BO',
    'BSOFT.BO', 'KPRMILL.BO', 'JKTYRE.BO'
]

# Function to fetch data for a single ticker
def fetch_data(ticker):
    try:
        stock = yf.Ticker(ticker)
        df = stock.history(start=start_date, end=end_date)
        if df.empty:
            print(f"No data found for {ticker}")
            return None
        df.reset_index(inplace=True)
        df['Ticker'] = ticker
        print(f"Fetched data for {ticker} - {len(df)} data points.")
        return df
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None

# Retry mechanism for fetching data
def fetch_data_with_retry(ticker, retries=2, delay=5):
    attempt = 0
    while attempt < retries:
        data = fetch_data(ticker)
        if data is not None and len(data) >= 200:  # Ensure enough data for MA200
            return data
        elif data is not None and len(data) < 200:
            print(f"Insufficient data for {ticker}: Only {len(data)} data points fetched.")
            return None
        else:
            print(f"Retrying {ticker} ({attempt+1}/{retries}) after {delay} seconds...")
            time.sleep(delay)
            attempt += 1
    print(f"Failed to fetch data for {ticker} after {retries} retries.")
    return None

# Fetch data concurrently with retry
def fetch_all_data(tickers):
    all_data = []
    valid_tickers = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:  # Optimized to 5 workers
        futures = {executor.submit(fetch_data_with_retry, ticker): ticker for ticker in tickers}
        for future in concurrent.futures.as_completed(futures):
            ticker = futures[future]
            data = future.result()
            if data is not None:
                all_data.append(data)
                valid_tickers.append(ticker)
            else:
                print(f"Excluding invalid or insufficient data ticker: {ticker}")
    return all_data, valid_tickers

print("Fetching stock data...")
all_data, valid_tickers = fetch_all_data(stock_tickers)
if not all_data:
    print("No data fetched. Please check the ticker symbols or network connection.")
else:
    print(f"Data fetched for {len(all_data)} tickers out of {len(stock_tickers)}.")

# Proceed only if data is fetched
if all_data:
    # Combine all data into a single DataFrame
    combined_df = pd.concat(all_data, ignore_index=True)

    # Function to generate signals for a single ticker
    def generate_signal(ticker, df):
        df = df.copy()  # Avoid SettingWithCopyWarning
        df.set_index('Date', inplace=True)

        # Calculate Moving Averages
        df['MA50'] = df['Close'].rolling(window=50).mean()
        df['MA200'] = df['Close'].rolling(window=200).mean()

        # Calculate RSI
        df['RSI'] = ta.rsi(df['Close'], length=14)

        # Calculate MACD
        macd = ta.macd(df['Close'], fast=12, slow=26, signal=9)
        df = df.join(macd)

        # Calculate Stochastic Oscillator
        stochastic = ta.stoch(df['High'], df['Low'], df['Close'], k=14, d=3, smooth_k=3)
        df = df.join(stochastic)

        # Drop rows with NaN values
        df.dropna(inplace=True)

        if df.empty:
            return {
                'Ticker': ticker,
                'Close_Price': None,
                'MA50': None,
                'MA200': None,
                'RSI': None,
                'MACD': None,
                'MACD_Signal': None,
                'Stochastic_K': None,
                'Stochastic_D': None,
                'Signal': 'Hold',
                'Reason': 'Insufficient data for indicators'
            }

        # Latest data point
        latest = df.iloc[-1]

        # Generate Signals with relaxed criteria
        signal = 'Hold'
        reason = ''
        target_price = None  # Initialize target price

        # Safely access 'Stochastic_K' and 'Stochastic_D'
        stochastic_k = latest.get('STOCHk_14_3_3', None)
        stochastic_d = latest.get('STOCHd_14_3_3', None)

        # Buy Signal Options:
        # 1. MA50 > MA200, RSI < 50, and MACD > MACD_Signal
        # 2. Stochastic_K < 20 and MACD > MACD_Signal
        if (latest['MA50'] > latest['MA200']) and (latest['RSI'] < 50) and (latest['MACD_12_26_9'] > latest['MACDs_12_26_9']):
            signal = 'Buy'
            # Simple target: 10% above current price
            target_price = round(latest['Close'] * 1.10, 2)
            reason = (f"The stock is in an uptrend, RSI indicates it is not oversold, "
                      f"and MACD shows positive momentum. Consider buying at the current price of ₹{latest['Close']:.2f}. "
                      f"Target to sell at ₹{target_price:.2f}.")
        elif (stochastic_k is not None and stochastic_k < 20) and (latest['MACD_12_26_9'] > latest['MACDs_12_26_9']):
            signal = 'Buy'
            # Simple target: 10% above current price
            target_price = round(latest['Close'] * 1.10, 2)
            reason = (f"The Stochastic Oscillator indicates the stock is oversold, and MACD shows positive momentum. "
                      f"Consider buying at the current price of ₹{latest['Close']:.2f}. "
                      f"Target to sell at ₹{target_price:.2f}.")

        # Sell Signal Options:
        # 1. MA50 < MA200, RSI > 50, and MACD < MACD_Signal
        # 2. Stochastic_K > 80 and MACD < MACD_Signal
        if (latest['MA50'] < latest['MA200']) and (latest['RSI'] > 50) and (latest['MACD_12_26_9'] < latest['MACDs_12_26_9']):
            signal = 'Sell'
            # Simple target: 10% below current price
            target_price = round(latest['Close'] * 0.90, 2)
            reason = (f"The stock is in a downtrend, RSI indicates it is not overbought, "
                      f"and MACD shows negative momentum. Consider selling at the current price of ₹{latest['Close']:.2f}. "
                      f"Target to buy back at ₹{target_price:.2f}.")
        elif (stochastic_k is not None and stochastic_k > 80) and (latest['MACD_12_26_9'] < latest['MACDs_12_26_9']):
            signal = 'Sell'
            # Simple target: 10% below current price
            target_price = round(latest['Close'] * 0.90, 2)
            reason = (f"The Stochastic Oscillator indicates the stock is overbought, and MACD shows negative momentum. "
                      f"Consider selling at the current price of ₹{latest['Close']:.2f}. "
                      f"Target to buy back at ₹{target_price:.2f}.")

        return {
            'Ticker': ticker,
            'Close_Price': latest['Close'],
            'MA50': latest['MA50'],
            'MA200': latest['MA200'],
            'RSI': latest['RSI'],
            'MACD': latest['MACD_12_26_9'],
            'MACD_Signal': latest['MACDs_12_26_9'],
            'Stochastic_K': stochastic_k,
            'Stochastic_D': stochastic_d,
            'Signal': signal,
            'Target_Price': target_price,
            'Reason': reason
        }

    # Generate signals for all valid tickers
    print("Performing technical analysis...")
    analysis_results = []
    for ticker in valid_tickers:
        df = combined_df[combined_df['Ticker'] == ticker]
        result = generate_signal(ticker, df)
        analysis_results.append(result)

    # Convert analysis results to DataFrame
    analysis_df = pd.DataFrame(analysis_results)

    # Define a fixed investment amount (e.g., ₹10,000 per Buy/Sell signal)
    investment_amount = 10000  # in INR

    # Function to calculate quantity to buy/sell
    def calculate_quantity(price, amount):
        if price and price > 0:
            return int(amount / price)
        else:
            return 0

    # Apply recommendations
    analysis_df['Recommended_Action'] = analysis_df['Signal']
    analysis_df['Recommended_Quantity'] = analysis_df.apply(
        lambda row: calculate_quantity(row['Close_Price'], investment_amount)
                    if row['Signal'] in ['Buy', 'Sell'] else 0,
        axis=1
    )

    # Separate Buy and Sell recommendations
    buy_recommendations = analysis_df[analysis_df['Signal'] == 'Buy']
    sell_recommendations = analysis_df[analysis_df['Signal'] == 'Sell']

    # Configure pandas display to show full 'Reason' text
    pd.set_option('display.max_colwidth', None)

    # Display Buy Recommendations
    print("\nBuy Recommendations:")
    if not buy_recommendations.empty:
        buy_display = buy_recommendations[['Ticker', 'Close_Price', 'RSI', 'Stochastic_K', 'Stochastic_D',
                                          'MACD', 'MACD_Signal', 'Recommended_Action', 'Recommended_Quantity',
                                          'Target_Price', 'Reason']]
        display(buy_display)
    else:
        print("No Buy signals generated based on the current criteria.")

    # Display Sell Recommendations
    print("\nSell Recommendations:")
    if not sell_recommendations.empty:
        sell_display = sell_recommendations[['Ticker', 'Close_Price', 'RSI', 'Stochastic_K', 'Stochastic_D',
                                            'MACD', 'MACD_Signal', 'Recommended_Action', 'Recommended_Quantity',
                                            'Target_Price', 'Reason']]
        display(sell_display)
    else:
        print("No Sell signals generated based on the current criteria.")

    # Display Summary of Signals
    print("\nSummary of Signals:")
    signal_counts = analysis_df['Signal'].value_counts()
    print(signal_counts)

    # Display Full Analysis
    print("\nFull Analysis:")
    display(analysis_df[['Ticker', 'Close_Price', 'MA50', 'MA200', 'RSI', 'Stochastic_K', 'Stochastic_D',
                        'MACD', 'MACD_Signal', 'Signal', 'Target_Price', 'Reason']])

Fetching stock data...
Fetched data for RELIANCE.NS - 492 data points.
Fetched data for HDFCBANK.NS - 492 data points.
Fetched data for TCS.NS - 492 data points.
Fetched data for ICICIBANK.NS - 492 data points.
Fetched data for INFY.NS - 492 data points.


ERROR:yfinance:$HDFC.NS: possibly delisted; no timezone found


Fetched data for SBILIFE.NS - 492 data points.
Fetched data for HINDUNILVR.NS - 492 data points.
Fetched data for BAJFINANCE.NS - 492 data points.
Fetched data for LT.NS - 492 data points.
No data found for HDFC.NS
Retrying HDFC.NS (1/2) after 5 seconds...
Fetched data for KOTAKBANK.NS - 492 data points.
Fetched data for ITC.NS - 492 data points.
Fetched data for ASIANPAINT.NS - 492 data points.
Fetched data for AXISBANK.NS - 492 data points.
Fetched data for BAJAJFINSV.NS - 492 data points.
Fetched data for ADANIPORTS.NS - 492 data points.
Fetched data for BAJAJ-AUTO.NS - 492 data points.
Fetched data for CIPLA.NS - 492 data points.
Fetched data for DRREDDY.NS - 492 data points.
Fetched data for DIVISLAB.NS - 492 data points.
Fetched data for JSWSTEEL.NS - 492 data points.
Fetched data for ULTRACEMCO.NS - 492 data points.
Fetched data for HDFCLIFE.NS - 492 data points.
Fetched data for HCLTECH.NS - 492 data points.
Fetched data for TECHM.NS - 492 data points.
Fetched data for TITAN.NS

ERROR:yfinance:$HDFC.BO: possibly delisted; no timezone found


Fetched data for HDFCBANK.BO - 475 data points.
Fetched data for INFY.BO - 475 data points.
Fetched data for ICICIBANK.BO - 475 data points.
Fetched data for HINDUNILVR.BO - 475 data points.
Fetched data for SBILIFE.BO - 475 data points.
Fetched data for LT.BO - 476 data points.
No data found for HDFC.BO
Retrying HDFC.BO (1/2) after 5 seconds...
Fetched data for KOTAKBANK.BO - 475 data points.
Fetched data for BAJFINANCE.BO - 475 data points.
Fetched data for ITC.BO - 475 data points.
Fetched data for AXISBANK.BO - 475 data points.
Fetched data for ASIANPAINT.BO - 475 data points.
Fetched data for BAJAJFINSV.BO - 475 data points.
Fetched data for ADANIPORTS.BO - 475 data points.
Fetched data for BAJAJ-AUTO.BO - 475 data points.
Fetched data for CIPLA.BO - 475 data points.
Fetched data for DRREDDY.BO - 476 data points.
Fetched data for JSWSTEEL.BO - 476 data points.Fetched data for DIVISLAB.BO - 475 data points.

Fetched data for HCLTECH.BO - 475 data points.
Fetched data for HDFCLIFE.B

...

ERROR:yfinance:$SPCL.BO: possibly delisted; no timezone found


No data found for SPCL.BO
Retrying SPCL.BO (2/2) after 5 seconds...
Failed to fetch data for HDFC.NS after 2 retries.
Excluding invalid or insufficient data ticker: HDFC.NS
Failed to fetch data for HDFC.BO after 2 retries.
Excluding invalid or insufficient data ticker: HDFC.BO
Failed to fetch data for SPCL.BO after 2 retries.
Excluding invalid or insufficient data ticker: SPCL.BO
Data fetched for 84 tickers out of 88.
Performing technical analysis...

Buy Recommendations:


,Ticker,Close_Price,RSI,Stochastic_K,Stochastic_D,MACD,MACD_Signal,Recommended_Action,Recommended_Quantity,Target_Price,Reason
17,DRREDDY.NS,6664.850098,46.751408,51.551979,52.135087,-22.522763,-25.966106,Buy,1,7331.34,"The stock is in an uptrend, RSI indicates it is not oversold, and MACD shows positive momentum. Consider buying at the current price of ₹6664.85. Target to sell at ₹7331.34."



Sell Recommendations:
No Sell signals generated based on the current criteria.

Summary of Signals:
Signal
Hold    83
Buy      1
Name: count, dtype: int64

Full Analysis:


,Ticker,Close_Price,MA50,MA200,RSI,Stochastic_K,Stochastic_D,MACD,MACD_Signal,Signal,Target_Price,Reason
0,RELIANCE.NS,2749.199951,2948.558701,2896.499344,32.454412,11.340293,8.420179,-50.673054,-26.875086,Hold,NaN,
1,HDFCBANK.NS,1633.150024,1659.929001,1560.051739,39.896423,12.000023,9.907855,-0.773983,14.417479,Hold,NaN,
2,TCS.NS,4252.950195,4373.386992,4026.374076,39.012532,19.520830,16.206722,-42.164706,-33.559204,Hold,NaN,
3,ICICIBANK.NS,1244.150024,1229.891003,1120.997277,45.163790,9.622626,7.532029,3.126422,13.820249,Hold,NaN,
4,INFY.NS,1952.750000,1884.868997,1635.188860,61.672466,71.993511,59.492264,12.935410,10.970422,Hold,NaN,
5,SBILIFE.NS,1737.400024,1805.777000,1554.701103,36.381337,5.613962,7.118013,-12.780753,7.066279,Hold,NaN,
6,HINDUNILVR.NS,2768.949951,2828.244004,2520.946862,36.520148,6.338495,12.958268,0.545213,26.845995,Hold,NaN,
7,BAJFINANCE.NS,7300.450195,7116.384023,6992.229456,47.210117,14.539128,14.100084,41.005444,111.808393,Hold,NaN,
8,LT.NS,3487.100098,3638.097988,3557.284091,37.019726,16.337244,12.295552,-36.396662,-6.770919,Hold,NaN,
9,KOTAKBANK.NS,1800.800049,1811.845005,1774.646536,43.185941,16.657013,12.689349,0.069265,12.317233,Hold,NaN,
